# MLP-based ANN (ANN-A) models.

In [1]:
import torch
import torch.optim as optim
import torch.nn as nn
import time
from torchtext import data
import random
import numpy as np
import os
from torch.nn import functional as F

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

### prepare data loader based on torchtext.

In [2]:
def get_iterator_feature(source_file, target_file, BATCH_SIZE=128):
    '''
    source_file: the source domain dataset in datasets/amazon_reivew/
    target file: the source domain dataset in datasets/amazon_reivew/
    '''
    TEXT = data.Field(dtype = torch.float,sequential=False, batch_first = True,use_vocab=False)
    LABEL = data.LabelField(dtype = torch.long,use_vocab=False)

    fields = {'text': ('text', TEXT), 'label': ('label', LABEL)}

    train_data = data.TabularDataset.splits(
                            path = 'datasets'+os.sep+"amazon_review",
                            train = source_file,
                            format = 'json',
                            fields = fields
    )
    test_data = data.TabularDataset.splits(
                            path = 'datasets'+os.sep+"amazon_review",
                            train = target_file,
                            format = 'json',
                            fields = fields
    )

    train_data = train_data[0]
    test_data = test_data[0]
    ## A very small (50) target labeled data is used to validate the model. You can set it to zeros. 
    test_data, valid_data = test_data.split(random_state = random.seed(SEED), split_ratio=0.95)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    source_iterator, target_iterator, valid_iterator = data.BucketIterator.splits(
        (train_data, test_data, valid_data), 
        batch_size = BATCH_SIZE, 
        sort=False,
        shuffle = True,
        # repeat=True,
        device = device)

    return source_iterator, target_iterator, valid_iterator

### Initialize ANN model.

In [3]:
from model.models import  ANNMLP
from model.criterion import MMD_loss

ann_version='ANN-A'

dataset = ['books_400.mat.json','dvd_400.mat.json','elec_400.mat.json','kitchen_400.mat.json']

source_file =dataset[0]
target_file = dataset[1]

source_iterator, target_iterator, valid_iterator = get_iterator_feature(source_file, target_file, BATCH_SIZE=128)

INPUT_DIM = 400
LATENT_DIM = 100
OUTPUT_DIM = 2
DROPOUT = 0.25
MU = 0.1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = ANNMLP(INPUT_DIM,LATENT_DIM,OUTPUT_DIM, DROPOUT, ann_version)

if ann_version == 'ANN':
    optimizer_task = optim.Adam(model.parameters())
else:
    optimizer_task = optim.Adam([{'params':model.extractor.parameters()},{'params':model.predictor.parameters()}])
    optimizer_kernel = optim.Adam([{'params':model.mmd_linear.parameters()},{'params':model.cmmd_linear.parameters()}])

criterion = nn.CrossEntropyLoss()
model = model.to(device)
criterion = criterion.to(device)
mmd_loss = MMD_loss(kernel_type='mmd', kernel_mul=2.0, kernel_num=5)
cmmd_loss = MMD_loss(kernel_type='cmmd', kernel_mul=2.0, kernel_num=5,eplison=0.00001)


### Training ANN (ANN-A) models.

In [4]:
from model.tools import train_adverisal,  train_normal

N_EPOCHS = 30
best_loss = 100.0
best_epoch = 0

for epoch in range(N_EPOCHS):

    start_time = time.time()
    if ann_version == 'ANN-A':
        train_loss = train_adverisal(model,source_iterator,target_iterator,optimizer_task,optimizer_kernel,criterion,mmd_loss,cmmd_loss)
    else:
        train_loss = train_normal(model,source_iterator,target_iterator,optimizer_task,criterion,mmd_loss,cmmd_loss,MU)


    eval_acc,eval_loss = evaluate(model, valid_iterator, criterion)
    if eval_loss < best_loss:
        best_loss = eval_loss
        best_epoch = epoch
        torch.save(model.state_dict(),'mmd-task-model.pt')

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)


    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s | Best Epoch:{best_epoch}')
    print(f'\tTrain Loss: {train_loss:.3f}|Valid Acc: {eval_acc:.3f}') 



RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.cuda.FloatTensor [50, 2]], which is output 0 of TBackward, is at version 2; expected version 1 instead. Hint: enable anomaly detection to find the operation that failed to compute its gradient, with torch.autograd.set_detect_anomaly(True).

### Test ANN models.

In [ ]:
from model.tools import evaluate

### test the model.
model.load_state_dict(torch.load('mmd-task-model.pt'))
eval_acc,eval_loss  = evaluate(model,target_iterator,criterion)
print('from %s to %s, acc is %f'%(source_file,target_file, eval_acc))